**Run the following two cells before you begin.**

In [1]:
%autosave 10

Autosaving every 10 seconds


In [2]:
import pandas as pd
import numpy as np

______________________________________________________________________
**First, import your data set and define the sigmoid function.**
<details>
    <summary>Hint:</summary>
    The definition of the sigmoid is $f(x) = \frac{1}{1 + e^{-X}}$.
</details>

In [3]:
# Import the data set
df = pd.read_csv('cleaned_data.csv')

In [4]:
# Define the sigmoid function
def sigmoid(x):
    y = 1/(1+np.exp(-x))
    return y

**Now, create a train/test split (80/20) with `PAY_1` and `LIMIT_BAL` as features and `default payment next month` as values. Use a random state of 24.**

In [5]:
# Create a train/test split
from sklearn.model_selection import train_test_split
X=df[['PAY_1','LIMIT_BAL']]
y=df['default payment next month']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24)

______________________________________________________________________
**Next, import LogisticRegression, with the default options, but set the solver to `'liblinear'`.**

In [6]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='liblinear')

______________________________________________________________________
**Now, train on the training data and obtain predicted classes, as well as class probabilities, using the testing data.**

In [7]:
# Fit the logistic regression model on training data
lr.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [8]:
# Make predictions using `.predict()`
pred=lr.predict(X_test)

In [28]:
# Find class probabilities using `.predict_proba()`
pred_proba=lr.predict_proba(X_test)

______________________________________________________________________
**Then, pull out the coefficients and intercept from the trained model and manually calculate predicted probabilities. You'll need to add a column of 1s to your features, to multiply by the intercept.**

In [29]:
# Add column of 1s to features
np.ones((X_test.shape[0],1))

ones_feat=np.hstack([np.ones((X_test.shape[0],1)),X_test])

In [30]:
# Get coefficients and intercepts from trained model
print('Coefficients Of Our Logistic Model',lr.coef_)
print('\n')
print('Intercept of Our Logistic Model',lr.intercept_)
lr_coeff=lr.coef_
lr_intercept=lr.intercept_

Coefficients Of Our Logistic Model [[ 8.27451187e-11 -6.80876727e-06]]


Intercept of Our Logistic Model [-6.57647457e-11]


In [31]:
coeff_interc_feats=np.concatenate([lr_intercept.reshape(1,1),lr_coeff],axis=1)
coeff_interc_feats
# Manually calculate predicted probabilities
man_pred_proba=np.dot(coeff_interc_feats,np.transpose(ones_feat))
man_pred_proba
def sigmoid_func(X):
    Y=1/(1/1+np.exp(-X))
    return Y
sigmoid_func(man_pred_proba)
sig_man_predproba=sigmoid_func(man_pred_proba)

______________________________________________________________________
**Next, using a threshold of `0.5`, manually calculate predicted classes. Compare this to the class predictions output by scikit-learn.**

In [32]:
# Manually calculate predicted classes
pred_manual=sig_man_predproba>=0.5
pred_manual.shape

(1, 5333)

In [33]:
# Compare to scikit-learn's predicted classes
np.array_equal(pred_manual,pred.reshape(1,-1))

True

______________________________________________________________________
**Finally, calculate ROC AUC using both scikit-learn's predicted probabilities, and your manually predicted probabilities, and compare.**

In [34]:
# Use scikit-learn's predicted probabilities to calculate ROC AUC
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,pred_proba[:,1])

0.627207450280691

In [35]:
# Use manually calculated predicted probabilities to calculate ROC AUC
roc_auc_score(y_test,man_pred_proba.reshape(man_pred_proba.shape[1],))

0.627207450280691